In [1]:
import os
import sys
# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)

In [2]:
from functions import write_dataframe_to_excel
from functions import read_and_check_input, Separate_data, nettoyer_dataframe
from functions import construire_tableau, Transpose_dataframe
from functions import format_constraints_elements, format_constraints_qualite, format_constraints_MP
from functions import solve_linear_program
from functions import construct_result_dataframe,Save_errors

In [3]:
def create_optimal_recipe(chemin_fichier):
    # Lire les fichiers Excel
    df_table, df_MP, df_contraints, erreurs = read_and_check_input(chemin_fichier)
    if len(erreurs) != 0 :
        return 0
    df_contraints_element, df_contraints_qualite, df_MP_dispo, df_MP_indispo = Separate_data(df_table, df_MP, df_contraints)

    # Suppression des matières premières indisponibles
    df_table = nettoyer_dataframe(df_table, df_MP_indispo)

    # Construction de la matrice A et du vecteur C
    A, C = construire_tableau(df_table, df_MP_dispo)

    # Initialisation des listes pour les contraintes
    constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

    df_contraints_element = Transpose_dataframe(df_contraints_element)
    # Mettre  les contraintes concernant les éléments
    constraints = format_constraints_elements(df_contraints_element, A,constraints)

    # Mettre  les contraintes concernant la qualité
    constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)

    # Mettre les contraintes concernant les matières premières disponibles
    constraints, bounds = format_constraints_MP(df_MP_dispo, constraints)

    # Résoudre le problème d'optimisation linéaire
    method = 'simplex' #'interior-point' 'simplex'
    res, ce_result, ci_result = solve_linear_program(C, constraints,bounds,method)
    conforme = all(val == 1 for val in ce_result) and  all(val == 1 for val in ci_result)    
    # print(method, '\n')
    # print(ce_result, ci_result)

    erreurs = []
    if not ce_result[0] :
        message = "Erreur : La proportion total n'est pas égale à 1"
        erreurs.append(message)

    # On recupere le chemin du dossier data
    dossier_data = os.path.dirname(chemin_fichier)
    if conforme :
        # Construire le DataFrame résultats
        df_res = construct_result_dataframe(df_MP_dispo, df_table, res)
        # Écrire le DataFrame résultats dans le fichier Excel
        write_dataframe_to_excel(df_res, dossier_data, new_sheet_name='Recette')
        print("Le problème admet une solution optimale.")
    else : 
        # Sauvegarder les erreurs dans un fichier texte
        message = "Veillez revoir les contraintes de la feuille 'Contraintes Qualités et Elément' et/ou 'Contraintes matières premières'"
        erreurs.append(message)
        Save_errors(erreurs, dossier_data, res, A, df_contraints_qualite, df_contraints_element, df_MP_dispo )
        print("Les erreurs ont été enregistrées dans le fichier erreurs.txt")
    return 


In [4]:
if __name__ == "__main__":
    # Chemin du fichier
    chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')
    create_optimal_recipe(chemin_fichier)

Les erreurs ont été enregistrées dans le fichier erreurs.txt


#### Essais


In [1]:
import os
import sys
# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)

from functions import write_dataframe_to_excel
from functions import read_and_check_input, Separate_data, nettoyer_dataframe
from functions import construire_tableau, Transpose_dataframe
from functions import format_constraints_elements, format_constraints_qualite, format_constraints_MP
from functions import solve_linear_program
from functions import construct_result_dataframe,Save_errors
import os

Resolution avec un correctif sur la borne maximale

In [52]:
def count_zeros(df_table):
    """
    Compte le nombre de zéros dans chaque colonne du dataframe.

    Cette fonction compte le nombre de zéros dans chaque colonne du dataframe donné.

    Paramètres :
        df_table (DataFrame) : Le dataframe dans lequel compter les zéros.

    Renvois :
        dict : Un dictionnaire contenant le compte des zéros pour chaque colonne.
    """
    zero_counts = {}
    for column in df_table.columns[1:]:
        zero_counts[column + '_max'] = (df_table[column] == 0).sum()
    return zero_counts

def reduce_b_max(b_ub, zero_counts, coeff):
    """
    Réduit la limite supérieure des contraintes en fonction du coefficient donné et du nombre de zéros.

    Cette fonction réduit la limite supérieure des contraintes en fonction du coefficient donné
    et du nombre de zéros dans chaque colonne, puis effectue une vérification entre les limites
    supérieures et inférieures pour s'assurer de leur cohérence.

    Paramètres :
        b_ub (dict) : Dictionnaire des limites supérieures des contraintes.
        zero_counts (dict) : Dictionnaire contenant le nombre de zéros pour chaque colonne.
        coeff (float) : Coefficient de réduction à appliquer.

    Returns:
        dict : Le dictionnaire des limites supérieures des contraintes réduit.
    """

    for key, _ in zero_counts.items():
        if key in b_ub :
            b_ub[key] *= coeff

    for key, value in b_ub.items():
        if key.endswith('_max') and key.replace('_max', '_min') in b_ub:
            min_key = key.replace('_max', '_min')
            if abs(b_ub[min_key]) > b_ub[key]:
                b_ub[key] = abs(b_ub[min_key])
    return b_ub

def reduce_b_max(b_ub, zero_counts, coeff):
    """
    Réduit la limite supérieure des contraintes en fonction du coefficient donné et du nombre de zéros.

    Cette fonction réduit la limite supérieure des contraintes en fonction du coefficient donné
    et du nombre de zéros dans chaque colonne, puis effectue une vérification entre les limites
    supérieures et inférieures pour s'assurer de leur cohérence.

    Paramètres :
        b_ub (dict) : Dictionnaire des limites supérieures des contraintes.
        zero_counts (dict) : Dictionnaire contenant le nombre de zéros pour chaque colonne.
        coeff (float) : Coefficient de réduction à appliquer.

    Returns:
        dict : Le dictionnaire des limites supérieures des contraintes réduit.
    """

    # Pour le Si et le C on prend la moyenne comme maximun
    b_ub['Si_max'] = (b_ub['Si_max'] + abs(b_ub['Si_min']))/2 
    b_ub['C_max'] = (b_ub['C_max'] + abs(b_ub['C_min']))/2 
    for key, _ in zero_counts.items():
        if key in b_ub and  (key not in ['Si_max','C_max']) :
            b_ub[key] *= coeff

    for key, value in b_ub.items():
        if key.endswith('_max') and key.replace('_max', '_min') in b_ub and  (key not in ['Si_max','C_max']):
            min_key = key.replace('_max', '_min')
            if abs(b_ub[min_key]) > b_ub[key]:
                b_ub[key] = abs(b_ub[min_key])
    return b_ub


def remove_max_key(zero_counts):
    """
    Supprime la clé avec la valeur maximale dans le dictionnaire zero_counts.

    Cette fonction supprime la clé avec la valeur maximale dans le dictionnaire zero_counts.
    Si le dictionnaire est vide, elle renvoie None.

    Paramètres :
        zero_counts (dict) : Dictionnaire contenant le nombre de zéros pour chaque colonne.

    Returns:
        dict or None : Le dictionnaire mis à jour sans la clé avec la valeur maximale, ou None si le dictionnaire est vide.
    """
    if not zero_counts:
        return None
    max_key = max(zero_counts, key=zero_counts.get)
    zero_counts.pop(max_key)
    return zero_counts


def find_errors(ce_result, ci_result, constraints,erreurs):
    """
    Trouve les erreurs dans les résultats des contraintes.

    Cette fonction vérifie les résultats des contraintes et ajoute des messages d'erreur
    aux erreurs existantes en cas de résultats incorrects.

    Paramètres :
        ce_result (array_like) : Résultats des contraintes d'égalité.
        ci_result (array_like) : Résultats des contraintes d'inégalité.
        constraints (dict) : Dictionnaire contenant les contraintes.
        erreurs (list) : Liste des erreurs existantes.

    Returns:
        list : Liste mise à jour des erreurs.
    """
    # Vérification des contraintes d'inégalité (b_ub)
    b_ub = constraints['b_sup']  
    b_ub_keys = list(b_ub.keys())
    ci_erros =  {}
    for key, value in zip(b_ub_keys, ci_result):
        if value == 0 and  key.endswith('_max') :
            ci_erros [key] = value
            error_name, error_value = key.split('_')
            message = f"Veuillez revoir la valeur {error_value} de {error_name}"
            erreurs.append(message)

    # Vérification des contraintes d'égalité (b_eq)
    b_eq = constraints['b_eq'] 
    b_eq_keys = list(b_eq.keys())
    ce_erros =  {}
    for key, value in zip(b_eq_keys, ce_result):
        if value == 0 :
            ce_erros [key] = b_eq [key]
            message = f"Veuillez revoir la valeur de '{key}' : {b_eq [key]}"
            erreurs.append(message)
    return erreurs


def optimize_with_coefficients(df_table, C, constraints, bounds, method, coefficients, erreurs):
    """
    Optimize with coefficients.

    This function optimizes with coefficients and returns the result and errors.

    Parameters:
        df_table (DataFrame): The dataframe table.
        C (array_like): Coefficients of the linear objective function.
        constraints (dict): Constraints of the linear optimization problem.
        bounds (tuple): Bounds of the decision variables.
        method (str): Method used for solving the linear optimization problem.
        coefficients (array_like): Coefficients for optimization.
        erreurs (list): List to store errors.

    Returns:
        tuple: Result and errors.

    """
    zero_counts = count_zeros(df_table)
    print(zero_counts)

    b_ub_true = constraints['b_sup']
    
    res = None
    while not erreurs:
        for coeff in coefficients:
            b_ub = b_ub_true.copy()
            constraints['b_sup'] = reduce_b_max(b_ub, zero_counts, coeff)
            res, ce_result, ci_result = solve_linear_program(C, constraints, bounds, method)
            erreurs = find_errors(ce_result, ci_result, constraints, erreurs)
            print(erreurs)
            print(constraints['b_sup'] )
            print(f"Méthode: {method}\nCoefficients: {coeff}\nRésultats CE: {ce_result}\nRésultats CI: {ci_result}\n")
            if not erreurs:
                break
            erreurs = []

        zero_counts = remove_max_key(zero_counts)
        if not zero_counts or not erreurs:
            print(" break not  zero_counts or not erreurs")
            break
        print(f"Colonnes restantes à vérifier : {zero_counts}")

    return res, erreurs


In [54]:

chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')

# Lire les fichiers Excel
df_table, df_MP, df_contraints, erreurs = read_and_check_input(chemin_fichier)

df_contraints_element, df_contraints_qualite, df_MP_dispo, df_MP_indispo = Separate_data(df_table, df_MP, df_contraints)

# Suppression des matières premières indisponibles
df_table = nettoyer_dataframe(df_table, df_MP_indispo)

# Construction de la matrice A et du vecteur C
A, C = construire_tableau(df_table, df_MP_dispo)

# Initialisation des listes pour les contraintes
constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

# Mettre  les contraintes concernant les éléments
constraints = format_constraints_elements(df_contraints_element, A,constraints)

# Mettre  les contraintes concernant la qualité
constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)

# Mettre les contraintes concernant les matières premières disponibles
constraints, bounds = format_constraints_MP(df_MP_dispo, constraints)

# Résoudre le problème d'optimisation linéaire
method = 'simplex' #'interior-point' 'simplex'
coefficients = [0.6, 0.65,0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
# coefficients = [0.6, 0.65,0.7]
res, erreurs = optimize_with_coefficients(df_table, C, constraints, bounds, method, coefficients, erreurs)


{'C_max': 3, 'Si_max': 3, 'Mn_max': 5, 'Cu_max': 6, 'Cr_max': 7, 'P_max': 5, 'Ni_max': 8, 'Mo_max': 9, 'Sn_max': 10, 'Sb_max': 9, 'Al_max': 6, 'S_max': 8, 'Mg_max': 9, 'Pb_max': 11, 'Ti_max': 9, 'As_max': 9, 'Bi_max': 11, 'V_max': 12}
["Veuillez revoir la valeur de 'Proportion_Total' : 1", "Veuillez revoir la valeur de '13 Retour Bleu' : 0.35"]
{'C_max': 3.5999999999999996, 'C_min': -3.55, 'Si_max': 2.05, 'Si_min': -1.9, 'Mn_max': 0.15, 'Mn_min': -0.15, 'Cu_max': 0.042, 'Cr_max': 0.03, 'P_max': 0.018, 'Ni_max': 0.009, 'Mo_max': 0.0036, 'Sn_max': 0.009, 'Sb_max': 0.003, 'Al_max': 0.010799999999999999, 'S_max': 0.018, 'Mg_max': 0.03, 'Pb_max': 0.018, 'Ti_max': 0.009, 'As_max': 0.003, 'Bi_max': 0.0009, 'V_max': 0.003, 'Impurete_max': 1.22, 'Impurete_min': -1.1, 'ONO_max': 0.2, 'ONO_min': -0.145}
Méthode: simplex
Coefficients: 0.6
Résultats CE: [0, 0]
Résultats CI: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0]

["Veuillez revoir la valeur de 'Proportion_Total'

In [55]:
constraints['b_sup']

{'C_max': 3.5999999999999996,
 'C_min': -3.55,
 'Si_max': 2.05,
 'Si_min': -1.9,
 'Mn_max': 0.2375,
 'Mn_min': -0.15,
 'Cu_max': 0.0665,
 'Cr_max': 0.0475,
 'P_max': 0.028499999999999998,
 'Ni_max': 0.014249999999999999,
 'Mo_max': 0.0057,
 'Sn_max': 0.014249999999999999,
 'Sb_max': 0.00475,
 'Al_max': 0.017099999999999997,
 'S_max': 0.028499999999999998,
 'Mg_max': 0.0475,
 'Pb_max': 0.028499999999999998,
 'Ti_max': 0.014249999999999999,
 'As_max': 0.00475,
 'Bi_max': 0.001425,
 'V_max': 0.00475,
 'Impurete_max': 1.22,
 'Impurete_min': -1.1,
 'ONO_max': 0.2,
 'ONO_min': -0.145}

In [ ]:

import pandas as pd
import numpy as np
from functions import Impurete_Values, ONO_Values

def calculate_results(res, df_MP_dispo, df_table):
    # Constantes pour les seuils
    SEUIL_0 = 1e-20
    SEUIL_1 = 1e-20  # 0.01

    # Création du DataFrame df_res
    df_res = df_MP_dispo[['Article', 'Prix', 'Métallique ?']].copy()

    # Ajout de la colonne 'Proportion' avec les valeurs de res.x
    df_res['Proportion'] = res.x

    # Calcul de la colonne 'Valeur (/T)' en multipliant 'Proportion' par 'Prix'
    df_res['Valeur (/T)'] = df_res['Proportion'] * df_res['Prix']

    # Filtre des lignes avec des proportions non nulles seulement
    df_res = df_res[df_res['Proportion'] > 0]

    # Tri du DataFrame par 'Proportion' de manière décroissante
    df_res.sort_values(by='Proportion', ascending=False, inplace=True)

    # Initialisation des colonnes 'ONO' et 'Impurete' avec des valeurs NaN
    df_res['ONO'] = np.nan
    df_res['Impurete'] = np.nan

    # Filtrage des lignes en fonction de la valeur de la colonne 'Métallique ?' et du seuil correspondant
    df_res = df_res.loc[((df_res['Métallique ?'] == 0) & (df_res['Proportion'] >= SEUIL_0)) |
                        ((df_res['Métallique ?'] == 1) & (df_res['Proportion'] >= SEUIL_1))]

    # Suppression de la colonne 'Métallique ?' qui n'est plus nécessaire
    df_res.drop(columns=['Métallique ?'], inplace=True)

    # Fusion avec les éléments chimiques correspondant aux articles sélectionnés
    articles_selectionnes = df_res['Article'].tolist()
    elements_selectionnes = df_table[df_table['Article'].isin(articles_selectionnes)]
    df_res = pd.merge(df_res, elements_selectionnes, on='Article', how='inner')


    # Ajout de la ligne de résultats
    df_res.loc[df_res.shape[0] , :] = np.nan
    df_res.loc[df_res.shape[0] , 'Article'] = 'Resultats'

    df_res.loc[df_res.shape[0] - 1, ['Proportion', 'Valeur (/T)']] = [df_res['Proportion'].sum(), df_res['Valeur (/T)'].sum()]

    # Calcul des proportions des éléments dans la fonte
    cols_elements = df_table.columns[1:]
    proportions_elements = df_res[cols_elements].mul(df_res['Proportion'], axis=0).sum()

    contraints_res = proportions_elements.to_dict()

    # Ajout des valeurs des proportions des éléments
    df_res.loc[df_res.shape[0]-1, cols_elements] = proportions_elements 

    # Calcul des valeurs des indicateurs qualité
    impurete_values, ono_values = np.array(list(Impurete_Values.values())), np.array(list(ONO_Values.values()))
    df_res.loc[df_res.shape[0]-1, 'Impurete'] = impurete_values @ proportions_elements
    df_res.loc[df_res.shape[0]-1, 'ONO'] = ono_values @ proportions_elements

    contraints_res['Impurete'] = impurete_values @ proportions_elements
    contraints_res['ONO'] = ono_values @ proportions_elements


    contraints_res['Proportion_Total'] = df_res.loc[df_res.shape[0] - 1, ['Proportion']]

    
    return df_res,contraints_res


In [56]:
df_res,contraints_res = calculate_results(res, df_MP_dispo, df_table)
contraints_res

{'C': 3.6000000000000023,
 'Si': 1.9874466402325273,
 'Mn': 0.2375000000000001,
 'Cu': 0.06650000000000003,
 'Cr': 0.047500000000000035,
 'P': 0.02364337501702383,
 'Ni': 0.009450654049869821,
 'Mo': 0.0053188832288361566,
 'Sn': 0.011402097843138323,
 'Sb': 0.0047500000000000025,
 'Al': 0.013607291513339745,
 'S': 0.02192183581126211,
 'Mg': 0.018570045233063128,
 'Pb': 0.0013912524703532298,
 'Ti': 0.006532117605964135,
 'As': 0.0009575855254198008,
 'Bi': 0.0011416398050955537,
 'V': 0.0034999999999999996,
 'Impurete': 1.099266952950228,
 'ONO': 0.16673263881318065,
 'Proportion_Total': Proportion    1.0
 Name: 9, dtype: object}

In [57]:
df_res

,Article,Prix,Proportion,Valeur (/T),ONO,Impurete,C,Si,Mn,Cu,...,Sn,Sb,Al,S,Mg,Pb,Ti,As,Bi,V
0,13 Retour Bleu,530,0.350000,185.5,NaN,NaN,3.65,2.640000,0.2200,0.0600,...,0.020000,0.01000,0.010000,0.010000,0.05000,0.000000,0.010000,0.001000,0.000000,0.0100
1,FDN0609 FONTE D'AFFINAGE,530,0.249613,132.294713,NaN,NaN,4.25,0.763000,0.0380,0.0010,...,0.001000,0.00300,0.003000,0.014000,0.00400,0.005000,0.011000,0.001000,0.004000,0.0000
2,FDN1207 E8C,453,0.226035,102.393839,NaN,NaN,0.10,0.020000,0.4500,0.0100,...,0.000000,0.00000,0.030000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
3,FDN0184 Disque,430,0.085688,36.84603,NaN,NaN,3.50,1.900000,0.4500,0.2000,...,0.000000,0.00000,0.000000,0.160000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
4,FDN1204. FRITE HAUT SILICIUM,507,0.071595,36.298448,NaN,NaN,0.10,3.000000,0.1500,0.1250,...,0.058000,0.00700,0.036000,0.017000,0.00100,0.002000,0.004000,0.005000,0.002000,0.0000
5,FDN1212 CARBONE 98-95 2-10MM,856,0.010355,8.864105,NaN,NaN,90.00,0.000000,0.0000,0.0000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
6,92655248 ferro silicum 75 en vrac,1520,0.006545,9.948496,NaN,NaN,0.00,75.000000,0.0000,0.0000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
7,FDN0263 Cuivre,7550,0.000169,1.276179,NaN,NaN,0.00,0.000000,0.0000,100.0000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0000
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Resultats,NaN,1.000000,513.421809,0.166733,1.099267,3.60,1.987447,0.2375,0.0665,...,0.011402,0.00475,0.013607,0.021922,0.01857,0.001391,0.006532,0.000958,0.001142,0.0035
